# Piping a Prompt, Model, and an Output Parser

In [1]:
%load_ext dotenv
%dotenv

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [4]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

In [5]:
list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [9]:
chat_template = ChatPromptTemplate.from_messages([('human',
                                                   "I've recently adopted a {pet}.Could you suggest three {pet} names?\n"+list_instructions)])

In [10]:
chat_template

ChatPromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template="I've recently adopted a {pet}. Could you suggest three {pet} names?\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`"), additional_kwargs={})])

In [11]:
print(chat_template.messages[0].prompt.template)

I've recently adopted a {pet}. Could you suggest three {pet} names?
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [25]:
chat = ChatOpenAI(model_name = "gpt-4",
                  model_kwargs = {"seed": 365},
                  temperature = 0,
                  max_tokens = 100)

C:\Users\Asus\.conda\envs\Langchain_env\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [26]:
list_output_parser = CommaSeparatedListOutputParser()

In [27]:
chat_value = chat_template.invoke({"pet": "dog"})

In [28]:
chat_value.messages[0].content

"I've recently adopted a dog. Could you suggest three dog names?\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`"

In [30]:
chat_result = chat.invoke(chat_value)

In [31]:
chat_result

AIMessage(content='Max, Bella, Charlie', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 49, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-C6F8qVjicI5ZJJ47cOqeiOMH6WSM1', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b1f6d261-e25a-47be-a121-3a1cc432d9bc-0', usage_metadata={'input_tokens': 49, 'output_tokens': 5, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [32]:
list_output_parser.invoke(chat_result)

['Max', 'Bella', 'Charlie']

In [33]:
chain = chat_template | chat | list_output_parser

In [34]:
chain.invoke({"pet": "dog"})

['Max', 'Bella', 'Charlie']